# Testing Graphical User Interfaces

In this chapter, we explore how to generate tests for Graphical User Interfaces (GUIs), abstracting from our [previous examples on Web testing](WebFuzzer.ipynb).  Building on general means to extract user interface elements and to activate them, our techniques generalize to arbitrary graphical user interfaces, from rich Web applications to mobile apps.

**Prerequisites**

* We build on the Web server introduced in the [chapter on Web testing](WebFuzzer.ipynb).

## Automated GUI Interaction

With our Web server: no JavaScript, no rich interfaces.  Also: Limited to Web.

How can we automate interaction?

### Our Web Server, Again

We (again) run our Web server.

In [ ]:
import fuzzingbook_utils

In [ ]:
from WebFuzzer import init_db, start_httpd, webbrowser, print_httpd_messages, print_url, ORDERS_DB

In [ ]:
db = init_db()

In [ ]:
httpd_process, httpd_url = start_httpd()
print_url(httpd_url)

In [ ]:
from IPython.core.display import display, Image
from fuzzingbook_utils import HTML

In [ ]:
HTML(webbrowser(httpd_url))

### Remote Control with Selenium

Let us just look at the GUI, above.  We do not assume we can access the HTML source, or even the URL of the current page.  All we assume is that there is a set of *user interface elements* we can interact with.

Documentation is available [here.](https://selenium-python.readthedocs.io/index.html)

In [ ]:
from selenium import webdriver

In [ ]:
from selenium.webdriver.firefox.options import Options

In [ ]:
options = Options()
options.headless = True

In [ ]:
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile

In [ ]:
ZOOM = 1.4
profile = FirefoxProfile()
profile.set_preference("layout.css.devPixelsPerPx", repr(ZOOM))

In [ ]:
driver = webdriver.Firefox(firefox_profile=profile, options=options)

In [ ]:
# Alternative: Chrome

# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=700x230')
# driver = webdriver.Chrome(options=options)

In [ ]:
def go_home(driver):
    driver.get(httpd_url)

In [ ]:
go_home(driver)

In [ ]:
print_httpd_messages()

In [ ]:
Image(driver.get_screenshot_as_png())

### Filling out Forms

In [ ]:
name = driver.find_element_by_name("name")
name.send_keys("Jane Doe")

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
email = driver.find_element_by_name("email")
email.send_keys("j.doe@example.com")

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
city = driver.find_element_by_name('city')
city.send_keys("Seattle")

In [ ]:
zip = driver.find_element_by_name('zip')
zip.send_keys("98104")

In [ ]:
terms = driver.find_element_by_name('terms')
terms.click()

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
submit = driver.find_element_by_name('submit')
submit.click()

In [ ]:
print_httpd_messages()

In [ ]:
Image(driver.get_screenshot_as_png())

### Navigating

In [ ]:
driver.back()

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
links = driver.find_elements_by_tag_name("a")

In [ ]:
links[0].get_attribute('href')

In [ ]:
links[0].click()

In [ ]:
print_httpd_messages()

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
driver.back()

In [ ]:
print_httpd_messages()

In [ ]:
Image(driver.get_screenshot_as_png())

## Retrieving UI Elements

In [ ]:
go_home(driver)

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
ui_elements = driver.find_elements_by_tag_name("input")

In [ ]:
for element in ui_elements:
    print(element.get_attribute('name'), element.get_attribute('type'), element.text)

In [ ]:
ui_elements = driver.find_elements_by_tag_name("button")

In [ ]:
for element in ui_elements:
    print(element.get_attribute('name'), element.get_attribute('type'), element.text)

In [ ]:
class GUIGrammarMiner(object):
    def __init__(self, driver):
        self.driver = driver
        self.grammar = {}

In [ ]:
class GUIGrammarMiner(GUIGrammarMiner):
    def mine_state_actions(self):
        actions = set()
        
        for elem in driver.find_elements_by_tag_name("input"):
            input_type = elem.get_attribute("type")
            input_name = elem.get_attribute("name")
            if input_name is None:
                input_name = elem.text

            if input_type in ["button", "checkbox", "radio"]:
                actions.add("click(%s)" % repr(input_name))
            elif input_type in ["text", "number", "password"]:
                actions.add("fill(%s, <quoted-%s>)" % (repr(input_name), input_type))
            elif input_type in ["submit"]:
                actions.add("submit(%s)" % repr(input_name))
            else:
                # TODO: Handle more types here
                actions.add("fill(%s, <quoted-%s>)" % (repr(input_name), input_type))

        for elem in driver.find_elements_by_tag_name("button"):
            button_type = elem.get_attribute("type")
            button_name = elem.get_attribute("name")
            if button_name is None:
                button_name = elem.text
            if button_type == "submit":
                actions.add("submit(%s)" % repr(input_name))
            elif button_type != "reset":
                actions.add("click(%s)" % repr(input_name))

        for elem in driver.find_elements_by_tag_name("a"):
            a_href = elem.get_attribute("href")
            if a_href is not None:
                actions.add("click(%s)" % repr(elem.text))

        return frozenset(actions)

\todo{Checkboxes can be clicked (toggled), but act as form values}

In [ ]:
gui_grammar_miner = GUIGrammarMiner(driver)
gui_grammar_miner.mine_state_actions()

This set of interactive elements makes up a _page_.

## Systematic GUI Exploration

### Representing States as Grammars

\todo{Have a generic interface `BaseGrammarMiner` with `__init__()` and `mine_grammar()`}

In [ ]:
from Grammars import new_symbol

In [ ]:
from Grammars import nonterminals, START_SYMBOL
from Grammars import extend_grammar, unreachable_nonterminals, opts, crange, srange
from Grammars import syntax_diagram

In [ ]:
from WebFuzzer import HTMLGrammarMiner

In [ ]:
class GUIGrammarMiner(GUIGrammarMiner):
    START_STATE = "<state>"
    FINAL_STATE = "<end>"
    UNEXPLORED_STATE = "<unexplored>"

    GUI_GRAMMAR = ({
        START_SYMBOL: ["<state>"],
        UNEXPLORED_STATE: [""],
        FINAL_STATE: [""],
        
        "<text>": ["<string>"],
        "<string>": ["<character>", "<string><character>"],
        "<character>": ["<letter>", "<digit>", "<special>"],
        "<letter>": crange('a', 'z') + crange('A', 'Z'),
        
        "<number>": ["<digits>"],
        "<digits>": ["<digit>", "<digits><digit>"],
        "<digit>": crange('0', '9'),
        
        "<special>": srange(". !"),

        "<email>": ["<letters>@<letters>"],
        "<letters>": ["<letter>", "<letters><letter>"],
        
        # Use a fixed password in case we need to repeat it
        "<password>": ["abcABC.123"],
    })

In [ ]:
syntax_diagram(GUIGrammarMiner.GUI_GRAMMAR)

In [ ]:
class GUIGrammarMiner(GUIGrammarMiner):
    for elem in ["text", "number", "email", "password"]:
        GUIGrammarMiner.GUI_GRAMMAR["<quoted-%s>" % elem] = \
            [("<%s>" % elem, opts(post=repr))]

In [ ]:
GUIGrammarMiner.GUI_GRAMMAR['<quoted-text>']

In [ ]:
class GUIGrammarMiner(GUIGrammarMiner):
    def new_state_symbol(self, grammar):
        return new_symbol(grammar, self.START_STATE)

    def mine_state_grammar(self, grammar=None, state_symbol=None):
        if grammar is None:
            grammar = extend_grammar(self.GUI_GRAMMAR)
        else:
            grammar = extend_grammar(grammar)

        if state_symbol is None:
            state_symbol = self.new_state_symbol(grammar)
            grammar[state_symbol] = []

        alternatives = []
        form = ""
        submit = None

        for action in self.mine_state_actions():
            if action.startswith("submit"):
                submit = action
                
            elif action.startswith("click"):
                link_target = self.new_state_symbol(grammar)
                grammar[link_target] = [self.UNEXPLORED_STATE]
                alternatives.append(action + '\n' + link_target)

            elif action.startswith("fill"):
                if len(form) > 0:
                    form += '\n'
                form += action

        if submit is not None:
            if len(form) > 0:
                form += '\n'
            form += submit

        if len(form) > 0:
            form_target = self.new_state_symbol(grammar)
            grammar[form_target] = [self.UNEXPLORED_STATE]
            alternatives.append(form + '\n' + form_target)
            
        alternatives += [self.FINAL_STATE]

        grammar[state_symbol] = alternatives
        
        # Remove unused parts
        for nonterminal in unreachable_nonterminals(grammar):
            del grammar[nonterminal]

        assert is_valid_grammar(grammar)
        
        return grammar

In [ ]:
gui_grammar_miner = GUIGrammarMiner(driver)
state_grammar = gui_grammar_miner.mine_state_grammar()
state_grammar

In [ ]:
state_grammar[GUIGrammarMiner.START_STATE]

The grammar actually encodes a Finite State Machine:

In [ ]:
from graphviz import Digraph
from IPython.display import display
from GrammarFuzzer import dot_escape
from collections import deque

In [ ]:
def fsm_diagram(grammar, start_symbol=START_SYMBOL):
    dot = Digraph(comment="Grammar as Finite State Machine")

    symbols = deque([start_symbol])
    symbols_seen = set()
    
    while len(symbols) > 0:
        symbol = symbols.popleft()
        symbols_seen.add(symbol)
        dot.node(symbol, dot_escape(symbol))
        
        for expansion in grammar[symbol]:
            nts = nonterminals(expansion)
            if len(nts) > 0:
                target_symbol = nts[-1]
                if target_symbol not in symbols_seen:
                    symbols.append(target_symbol)

                label = expansion.replace(target_symbol, '')
                dot.edge(symbol, target_symbol, label.replace('\n', r'\l'))
                
    display(dot)

In [ ]:
fsm_diagram(state_grammar)

In [ ]:
# Remove unused parts
for nonterminal in unreachable_nonterminals(state_grammar):
    del state_grammar[nonterminal]

In [ ]:
from GeneratorGrammarFuzzer import GeneratorGrammarFuzzer

In [ ]:
gui_fuzzer = GeneratorGrammarFuzzer(state_grammar)
print(gui_fuzzer.fuzz())

### Fuzzing GUI Forms

In [ ]:
from Fuzzer import Runner

In [ ]:
class GUIRunner(Runner):
    def __init__(self, driver):
        self.driver = driver
        
    def run(self, inp):
        def click(name):
            self.do_click(name)
        def fill(name, value):
            self.do_fill(name, value)
        def submit(name):
            self.do_submit(name)
        
        exec(inp, {}, {'click': click, 'submit': submit, 'fill': fill})
        
        return inp, self.PASS

In [ ]:
from selenium.common.exceptions import NoSuchElementException

In [ ]:
class GUIRunner(GUIRunner):
    def find_element(self, name):
        try:
            return self.driver.find_element_by_name(name)
        except NoSuchElementException:
            return self.driver.find_element_by_link_text(name)

We need delays (in seconds):

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
class GUIRunner(GUIRunner):
    DELAY_AFTER_CLICK = 1
    DELAY_AFTER_SUBMIT = 1
    DELAY_AFTER_FILL = 0.1

In [ ]:
class GUIRunner(GUIRunner):
    def do_click(self, name):
        element = self.find_element(name)
        element.click()
        WebDriverWait(self.driver, self.DELAY_AFTER_CLICK)

In [ ]:
class GUIRunner(GUIRunner):
    def do_submit(self, name):
        element = self.find_element(name)
        element.click()
        WebDriverWait(self.driver, self.DELAY_AFTER_SUBMIT)

In [ ]:
class GUIRunner(GUIRunner):
    def do_fill(self, name, value):
        element = self.find_element(name)
        element.send_keys(value)
        WebDriverWait(self.driver, self.DELAY_AFTER_FILL)

In [ ]:
go_home(driver)

In [ ]:
gui_runner = GUIRunner(driver)

In [ ]:
gui_runner.run("fill('name', 'Walter White')")

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
gui_runner.run("submit('submit')")

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
go_home(driver)

In [ ]:
gui_fuzzer = GeneratorGrammarFuzzer(state_grammar)

In [ ]:
while True:
    action = gui_fuzzer.fuzz()
    if action.find('submit(') > 0:
        break

In [ ]:
print(action)

In [ ]:
gui_runner.run(action)

In [ ]:
Image(driver.get_screenshot_as_png())

### Exploring States

In [ ]:
import time

In [ ]:
from Grammars import is_nonterminal, is_valid_grammar

In [ ]:
from GeneratorGrammarFuzzer import ProbabilisticGeneratorGrammarCoverageFuzzer as PGGCFuzzer

In [ ]:
class GUIFuzzer(PGGCFuzzer):
    def __init__(self, driver, restart=None, **kwargs):
        self.driver = driver
        self.miner = GUIGrammarMiner(driver)
        self.restart_function = restart

        self.states_seen = set()
        self.state_symbol = GUIGrammarMiner.START_STATE
        self.state = self.miner.mine_state_actions()
        self.states_seen.add(self.state)
        
        grammar = self.miner.mine_state_grammar()
        super().__init__(grammar, **kwargs)
        
    def restart(self):
        if self.restart_function is not None:
            self.restart_function(driver)
        else:
            # Generic back to start
            for i in range(20):
                self.driver.back()

        self.state = GUIGrammarMiner.START_STATE
            
    def fsm_path(self, tree):
        """Return sequence of state symbols"""
        (node, children) = tree
        if node == GUIGrammarMiner.UNEXPLORED_STATE:
            return []
        elif children is None or len(children) == 0:
            return [node]
        else:
            return [node] + self.fsm_path(children[-1])

    def fsm_last_state_symbol(self, tree):
        """Return current (expected) state symbol"""
        for state in reversed(self.fsm_path(tree)):
            if is_nonterminal(state):
                return state
        assert False

    def run(self, gui_runner):
        assert isinstance(gui_runner, GUIRunner)
        
        self.restart()
        action = self.fuzz()
        self.state_symbol = self.fsm_last_state_symbol(self.derivation_tree)

        print(action, self.state_symbol)
        
        result, outcome = gui_runner.run(action)

        # FIXME: Need to wait until load is complete
        time.sleep(1)
        
        self.update_states()

        return self.state_symbol, outcome

    def update_states(self):
        self.state = self.miner.mine_state_actions()
        if self.state in self.states_seen:
            return
        
        print("Found new state", self.state)
        self.states_seen.add(self.state)

        state_grammar = self.miner.mine_state_grammar(grammar=self.grammar, 
                                                      state_symbol=self.state_symbol)
        del state_grammar[START_SYMBOL]
        del state_grammar[GUIGrammarMiner.START_STATE]
        print("State grammar:", state_grammar)

        new_grammar = extend_grammar(self.grammar, state_grammar)
        self.grammar = new_grammar  # FIXME: Should be set_grammar or likewise

In [ ]:
go_home(driver)

In [ ]:
gui_fuzzer = GUIFuzzer(driver, restart=go_home)

In [ ]:
gui_fuzzer.fuzz()

In [ ]:
from GrammarFuzzer import display_tree

In [ ]:
gui_fuzzer.run(gui_runner)

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
gui_fuzzer.restart()

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
gui_fuzzer.grammar

In [ ]:
fsm_diagram(gui_fuzzer.grammar)

\todo{Consider an alternate encoding that does not need a generator}

In [ ]:
for i in range(10):
    print("Run", i)
    gui_fuzzer.run(gui_runner)

In [ ]:
fsm_diagram(gui_fuzzer.grammar)

In [ ]:
gui_fuzzer.missing_expansion_coverage()

### Optimizations

\todo{Special challenge: We need to go back to get to earlier state.}

What is in unknown?

## Fun with FuzzingBook

\todo{Create a full map of fuzzingbook.org, only by navigating}

In [ ]:
driver.get("https://www.fuzzingbook.org/")

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
pass
# links = driver.find_elements_by_tag_name("a")
# for link in links:
#     print(link.tag_name, link.text, link.get_attribute("href"))

That's it – we're done!

In [ ]:
driver.quit()

In [ ]:
httpd_process.terminate()

In [ ]:
import os

In [ ]:
for temp_file in [ORDERS_DB, "geckodriver.log", "ghostdriver.log"]:
    if os.path.exists(temp_file):
        os.remove(temp_file)

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```markdown
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_